# Working with FITS image data

In this section we will learn how access image data.

# High-level interface for individual images

Our introduction to working with FITS image data uses a high-level interface that avoids the need to understand how to set up a number of useful astropy objects, like a WCS. The `CCDData` object takes care of those details for you but is harder to use with more complex FITS files

One note about `CCDData` -- it requires that the image have a unit. In the example we've included the FITS file itself does not include a unit for the data so we provide that below

In [ ]:
from astropy.nddata import CCDData

In [ ]:
ccd = CCDData.read("pix.fits.gz", unit="adu")

### Quick visualization example

We will return to amore extensive discussion of visualization shortly. Below we demonstrate one way display an image.

In [ ]:
from astropy.visualization import imshow_norm, PercentileInterval, LogStretch
from matplotlib import pyplot as plt

%matplotlib inline

For astronomical images, you can use `astropy.visualization` to [normalize and stretch](https://docs.astropy.org/en/stable/visualization/normalization.html) the display. (`astropy.visualization` will be covered more in a later section.) Here, we want it to be logarithmic.

In [ ]:
# Display the image (you can ignore the warnings if you see them)
fig, ax = plt.subplots()
im, norm = imshow_norm(
    ccd.data, ax, origin='lower',
        interval=PercentileInterval(99.9), stretch=LogStretch())
fig.colorbar(im)

### A few `CCDData` properties

The `header` attribute contains the FITS header for the image.

In [ ]:
ccd.header

You can get the data itself with the `.data` attribute

In [ ]:
ccd.data

If the FITS file contains WCS information that is available via the `.wcs` attribute.

In [ ]:
ccd.wcs

In [ ]:
ccd.wcs.pixel_to_world((10, 256, 50), (256, 256, 480))

### Writing FITS files

The `CCDData` object has a `write` method for writing FITS files. It recognizes a broad range of extensions, and can write compressed files if desired.

In [ ]:
ccd.write("test_write.fts")

### Error and masking

You can carry error and masking information along with images data in a `CCDData` object. Here we create the error by simply assuming the only source of error in the image is Poisson (that is not the case).

We import the `numpy` module to get an array-aware square root.

In [ ]:
import numpy as np
ccd.uncertainty = np.sqrt(ccd.data)

It appears that there were some negative values in the data, leading to the invalid value warning above. Let's also create a mask that indicates which pixels should be ignored.

Note that a value of `True` in the mask means the pixel should be ignored.

Also note that *not every function, method, etc pays attention to the mask*. This is true of both astropy and other packages.

In [ ]:
ccd.mask = ccd.data < 0
ccd.mask.sum()

### Saving this "compound" image

We can save this image the same way we did before. As we will see in a moment, the error and mask are saved to separate extensions.

If you wish to overwrite an existing file you must use the `overwrite=True` argument.

In [ ]:
ccd.write("pix_with_error_mask.fits", overwrite=True)

## Low-level interface for reading FITS files

The `fits` module provides an interface for working with FITS files that are more complex than a single image.

We will explore this using the image we just saved.

In [ ]:
from astropy.io import fits

The `open()` function in [astropy.io.fits](https://docs.astropy.org/en/stable/io/fits/index.html) works with regular and compressed files.

We begin by opening this FITS file and looking at its structure.

In [ ]:
with fits.open('pix_with_error_mask.fits') as f:
    f.info()

### Reading specific extensions

You can get information from specific extensions by referring to them by name or number. The example below reads in just the mask HDU, which is extension 1.

In [ ]:
with fits.open("pix_with_error_mask.fits") as f:
    mask_hdu = f[1]

You can use the `header` attribute of the HDU to access the header:

In [ ]:
mask_hdu.header

You might expect that you can use the `.data` attribute to access the data, but that does not work:

In [ ]:
mask_hdu.data

#### Explanation

What just happened?! It certainly appeared as if the mask HDU had been loaded since we could access its header, but the data clearly was not loaded. 

The error message, `ValueError: I/O operation on closed file`, provides a clue.

The reason is primarily historical. FITS files are, by default, opened in *memory map* mode to avoid reading in the entire file at once. In memory map mode, the `.data` that looks like an array is actually, because of the way numpy handles memory maps, a reference to the file with the data. That has been the default in astropy since its very beginning, including in the stand-alone `pyfits` package that preceded, and then became a part of, astropy. 


On the other hand, a context manager is supposed to guarantee that any resources created in the context, like opening a file handle, is cleaned up, i.e. removed, when you exit the context.

That has led to a compromise:

1. If no reference is made to the data in the with block, e.g., by assigning the data to a variable, and the file is opened with `memmap=True`, the default, then all file handles are closed when you exit the block and you have no access to the data after exiting the block. You do have access to the header because that is always read from disk. This is the case we saw above.
2. If there is a reference to the data in the `with` block, and the file is opened with `memmap=True`, the default, then a reference to the file is maintained by `numpy`, which handles the memory map for astropy. That gives you access to the memory-mapped data at the expense of maintaining a hidden file reference. An example of this is below. 
3. If `memmap=False` the situation is less confusing because the header and data are read in as needed. However, the data is not read in unless it is accessed inside the with block. The downside is that more memory is used, which can be an issue for large images. An example of this is below.


In [ ]:
# Case 2 -- explicitly reference the data
with fits.open("pix_with_error_mask.fits") as f:
    mask_hdu = f[1]
    mask_hdu_data = mask_hdu.data

# We have access to the data via a "hidden" file reference
# to "pix_with_error_mask.fits"
mask_hdu_data[0, 0]

In [ ]:
# Case 3 -- no memory mapping
with fits.open("pix_with_error_mask.fits", memmap=False) as f:
    mask_hdu = f[1]
    # This causes the data to be read from disk, so it is accessible outside
    # the with block. Deleting this line will make this cell fail because the 
    # data is only read if the user explicitly accesses it.
    mask_hdu.data

# We have access to the data because it was read into memory
mask_hdu.data[0, 0]

## Working with headers

Headers are similar to Python dictionaries. Here, we will look at a header, modify an existing keyword, and add a new card. Note that the FITS file is opened in `update` mode, which means that any changes made to the header or data are saved back to the file.

In [ ]:
keyname = 'CRPIX1'
extnum = 1

with fits.open('j94f05bgq_flt.fits.gz', mode='update') as f:
    value = f[extnum].header.get(keyname)  # None if non-existent
    print("{}: {}".format(keyname, value))
    f[extnum].header[keyname] = value + 1  # Reassign the keyword
    print("Updated {}: {}".format(keyname, f[extnum].header[keyname]))

Note that once we leave with `with` block changes no longer update the file. Also, unlike regular Python dictionaries, FITS headers are not case-sensitive.

In [ ]:
# FITS header keyword is also not case-sensitive.
f[extnum].header['observer'] = "Henrietta Leavitt"
print('observer:', f[extnum].header['OBSERVER'])


To delete a keyword/card, it is best to use the *remove* method, which does not raise an exception if the keyword is not present and `ignore_missing=True` option is provided.

In [ ]:
f[extnum].header.remove('observer', ignore_missing=True)

Comment and history cards are added with special methods. In this case, a new card is always created.

In [ ]:
import time

with fits.open('j94f05bgq_flt.fits.gz', mode='update') as f:
    f[extnum].header.add_history('{} New history card.'.format(time.ctime()))
    f[extnum].header.add_comment('This is a cool image.')
    f[extnum].header.add_comment('Much science. So Python.')
    print(f[extnum].header['comment'])
    print()
    print(f[extnum].header['history'])

## Working with image data

In [ ]:
with fits.open('pix.fits.gz') as f:
    f.info()
    scidata = f[0].data

An image is a NumPy array saved as the data part of an HDU.

In [ ]:
print(scidata.shape)
print(scidata.dtype)

👀 `scidata` is a copy of the data array of the HDU. If it changes, it will not affect the data in the FITS file.

In [ ]:
# All operations available to NDArray are applicable to the FITS data array.
scidata[2:10, 3:7].mean()

## Working with FITS tables

**Note**: The recommended method to read and write a single FITS table is using the [Unified I/O read/write interface](https://docs.astropy.org/en/stable/io/unified.html#table-io-fits):

    from astropy.table import Table
    t = Table.read('data.fits')

We also show an example of using [astropy.io.fits](https://docs.astropy.org/en/stable/io/fits/index.html#) below as there is a lot of legacy code which uses it:

    with fits.open('data.fits') as hdu_list:
        hdu_list.info()
        table_data = hdu_list[1].data
        print('Column names: \n', table_data.names)
        print('\nRow 1: \n', table_data[1])
        print('\nColumn "time": \n', table_data.field('time'))
        print('\nNumber of rows: \n', len(table_data))